**analysis.ipynb**

**Description:** Interactive auxillary support for my summer internship repport. This is the analysis of three recurrent neural networks using mainly primary component analysis (PCA) and slow point analysis.

**Author:** Aude Forcione-lambert, University of Montreal, Faculty of Arts and Sciences

**Email:** aude.forcione-lambert@umontreal.ca

# Importations

In [1]:
import numpy as np
import scipy
from sklearn.decomposition import PCA

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D

In [2]:
# Uncomment for interactive graphics. Be aware that this will slow down the program.
%matplotlib notebook

# Useful variables

In [3]:
# Number of neurons
nb_neurons = 100
# Identity matrix matching the main matrix's size
I = np.diag(np.ones(nb_neurons))
# Possible switches by index
# Possible outputs are either represented by tensions (3 channels of real numbers roughly between -1 and 1) or by an
# index between 0 and 8. Index and tension of each three channels are related by the binary representation of the
# index. The unit digit corresponds to channel one, a 0 digit corresponds to a tension of -1 and a 1 to a tension of 1.
switch = np.array([[0,1],[0,2],[0,4],[1,3],[1,5],[2,3],[2,6],[3,7],[4,5],[4,6],[5,7],[6,7]])
# Colors used in graphics for each index
colors = ('indianred','darkorange','gold','lime','turquoise','royalblue','blueviolet','magenta')

# Useful functions

In [4]:
# Converts channel tensions to the corresponding index... Honestly it's pretty self-explanatory............................. Do I really have to do this everywhere? I'm not using three meters long variable names for fun.............................................................. Fine, I'll do it. This is for you, all my past programming teachers........................ Shoutout to Sonia Gounar, you are one great teacher, even if you did always remove some of my points for non-sufficient comments... I might have deserved it.............................................................................................................................................................................................................................I need coffee
def tension_to_index(tension):
    tension = np.round(tension)
    return int((tension[0]+1)/2+(tension[1]+1)+2*(tension[2]+1))

# Same but the other way around
def index_to_tension(index):
    tension = np.zeros((3))
    tension[0] = (index%2)*2-1
    # These >> are bitshifts. Bitshifts are cool
    tension[1] = ((index >> 1)%2)*2-1
    tension[2] = ((index >> 2)%2)*2-1
    return tension

# Calculates next network state in the absence of input. Useful for q function
def F(state,w):
    return np.tanh(np.matmul(state,w))

# Energy function used for finding slow points
def q(state,w):
    return 1/2.0*np.linalg.norm(F(state,w)-state)**2

# Find all instances of a short pattern in a long array
def matchPattern(base_array,pattern):
    indexes = []
    for i in range(len(base_array)-len(pattern)):
        if np.all(base_array[i:i+len(pattern)] == pattern):
            indexes.append(i)
    return np.array(indexes)

# Data importation

In [5]:
# Input array  (3 channels)
x_test_array = np.loadtxt('Data/x_test_array.gz')
# Expected output array (3 channels)
y_test_array = np.loadtxt('Data/y_test_array.gz')

# Loss over time for random_seed = 7 and time_delay = 0
#s7d0_loss = np.loadtxt('Data/s7d0_loss.gz')
# Output during testing
s7d0_output = np.loadtxt('Data/s7d0_output.gz')
# States of neurons during testing
s7d0_states = np.loadtxt('Data/s7d0_states.gz')
# W matrix (or strengths of neurons connections)
s7d0_w = np.loadtxt('Data/s7d0_w.gz')
# Wout matrix
s7d0_wout = np.loadtxt('Data/s7d0_wout.gz')

# Same for random_seed = 7 and time_delay = 4
s7d4_loss = np.loadtxt('Data/s7d4_loss.gz')
s7d4_output = np.loadtxt('Data/s7d4_output.gz')
s7d4_states = np.loadtxt('Data/s7d4_states.gz')
s7d4_w = np.loadtxt('Data/s7d4_w.gz')
s7d4_wout = np.loadtxt('Data/s7d4_wout.gz')

# Same for random_seed = 7 and time_delay = 8
#s7d8_loss = np.loadtxt('Data/s7d8_loss.gz')
s7d8_output = np.loadtxt('Data/s7d8_output.gz')
s7d8_states = np.loadtxt('Data/s7d8_states.gz')
s7d8_w = np.loadtxt('Data/s7d8_w.gz')
s7d8_wout = np.loadtxt('Data/s7d8_wout.gz')

# Win matrix for random_seed = 7 and all time delays
#s7_win = np.loadtxt('Data/s7_win.gz')

# Task Definition

In [6]:
fig,(ax) = plt.subplots(1,1,sharex=True,figsize=(8,2.5))
# Expected output
ax.plot(y_test_array[:500,0],'k')
# Input
ax.plot(x_test_array[:500,0],'r')

ax.set_ylabel('Tension')
ax.set_xlabel('Elapsed time steps')
custom_legend = [Line2D([0], [0], color='r', label='Input'),
                Line2D([0], [0], color='k', label='Expected output')]
ax.legend(handles=custom_legend, loc='upper center', bbox_to_anchor=(.5,1.3), ncol=2)

fig.tight_layout()

<IPython.core.display.Javascript object>

---
Basic definition of the task. The neural network is expected to match and maintain the last input of one or minus one.

---

In [7]:
fig,(ax) = plt.subplots(1,1,sharex=True,figsize=(8,2.5))
# Setting delay for the demonstration's purpose
delay = 7
# Expected output
ax.plot(y_test_array[:500,0],'k')
# Input
ax.plot(x_test_array[delay:500+delay,0],'r')
# Visualisation of delay
ax.set_yticks([-1,0,1])
for i in np.where(np.abs(np.round((y_test_array[0:499,0]-x_test_array[1:500,0])))==2)[0] :
    ax.axvspan(i-delay+1,i,color='lightgray')

ax.set_ylabel('Tension')
ax.set_xlabel('Elapsed time steps')
custom_legend = [Line2D([0], [0], color='r', label='Input'),
                Line2D([0], [0], color='k', label='Expected output'),
                mpl.patches.Patch(facecolor='lightgray', edgecolor='lightgray', label='Duration of delay')]
ax.legend(handles=custom_legend, loc='upper center', bbox_to_anchor=(.5,1.3), ncol=3)

fig.tight_layout()

<IPython.core.display.Javascript object>

---
New task definition. Now the neural network must lag (and therefore remember) during a certain number of time steps after the input.

---

# Training

In [8]:
plt.figure()
plt.subplot(1,1,1)
plt.xlabel("Epoch index")
plt.ylabel("Mean squared error")
plt.plot(np.arange(3000),s7d4_loss[:3000])

<IPython.core.display.Javascript object>

---
Sample loss over training epoch. This graphic was generated during training of the RNN with a random seed of 7 and a delay of 4 time-steps.

---

# Elementary case

In [9]:
# Simple pca analysis of the test for random_seed=7 and time_delay=0

s7d0_pca = PCA(n_components=3)
s7d0_pca.fit(s7d0_states)
s7d0_pc1, s7d0_pc2, s7d0_pc3 = s7d0_pca.transform(s7d0_states).T

# Type of output for the neurons states
s7d0_out_type = np.array([tension_to_index(out) for out in s7d0_output])

In [10]:
# Drawing of pca analysis

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

#
for i in range(3000):
    ax.scatter(s7d0_pc1[i], s7d0_pc2[i], s7d0_pc3[i], c=colors[s7d0_out_type[i]], alpha=0.05)

ax.plot(s7d0_pc1[:3000], s7d0_pc2[:3000], s7d0_pc3[:3000], '-k', linewidth=0.5, alpha=0.5)

custom_legend = [
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([1,1,1])], marker = '.', label = 'Out (1,1,1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,1,1])], marker = '.', label = 'Out (-1,1,1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([1,-1,1])], marker = '.', label = 'Out (1,-1,1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,-1,1])], marker = '.', label = 'Out (-1,-1,1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([1,1,-1])], marker = '.', label = 'Out (1,1,-1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,1,-1])], marker = '.', label = 'Out (-1,1,-1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([1,-1,-1])], marker = '.', label = 'Out (1,-1,-1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,-1,-1])], marker = '.', label = 'Out (-1,-1,-1)'),
    Line2D([0], [0], color = 'gray', label = 'Path of output')
]
ax.legend(handles = custom_legend, loc=9, ncol=3)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.view_init(8,-63)

<IPython.core.display.Javascript object>

---
PCA analysis for the neurons states during test. The points at each time steps are color-coded in respect to the effective output value corresponding to the network state.

---

# Adding delay

Delay = 4

In [11]:
# Simple pca analysis of the test for random_seed=7 and time_delay=4

s7d4_pca = PCA(n_components=3)
s7d4_pca.fit(s7d4_states)
s7d4_pc1, s7d4_pc2, s7d4_pc3 = s7d4_pca.transform(s7d4_states).T

# Type of output for the neurons states
s7d4_out_type = np.array([tension_to_index(out) for out in s7d4_output])

In [12]:
# Drawing of pca analysis

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

#
for i in range(3000):
    ax.scatter(s7d4_pc1[i], s7d4_pc2[i], s7d4_pc3[i], c=colors[s7d4_out_type[i]], alpha=0.05)

ax.plot(s7d4_pc1[:3000], s7d4_pc2[:3000], s7d4_pc3[:3000], '-k', linewidth=0.5, alpha=0.5)

custom_legend = [
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([1,1,1])], marker = '.', label = 'Out (1,1,1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,1,1])], marker = '.', label = 'Out (-1,1,1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([1,-1,1])], marker = '.', label = 'Out (1,-1,1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,-1,1])], marker = '.', label = 'Out (-1,-1,1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([1,1,-1])], marker = '.', label = 'Out (1,1,-1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,1,-1])], marker = '.', label = 'Out (-1,1,-1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([1,-1,-1])], marker = '.', label = 'Out (1,-1,-1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,-1,-1])], marker = '.', label = 'Out (-1,-1,-1)'),
    Line2D([0], [0], color = 'gray', label = 'Path of output')
]
ax.legend(handles = custom_legend, loc=9, ncol=3)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.view_init(8,-48)

<IPython.core.display.Javascript object>

---
PCA analysis of test for delay of 4 time-steps.

---

Delay = 8

In [13]:
# Simple pca analysis of the test for random_seed=7 and time_delay=8

s7d8_pca = PCA(n_components=3)
s7d8_pca.fit(s7d8_states)
s7d8_pc1, s7d8_pc2, s7d8_pc3 = s7d8_pca.transform(s7d8_states).T

# Type of output for the neurons states
s7d8_out_type = np.array([tension_to_index(out) for out in s7d8_output])

In [14]:
# Drawing of pca analysis

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

# Color coded points for neurons' states at each time step
for i in range(3000):
    ax.scatter(s7d8_pc1[i], s7d8_pc2[i], s7d8_pc3[i], c=colors[s7d8_out_type[i]], alpha=0.05)
# Path over time of neurons' states
ax.plot(s7d8_pc1[:3000], s7d8_pc2[:3000], s7d8_pc3[:3000], '-k', linewidth=0.5, alpha=0.5)

custom_legend = [
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([1,1,1])], marker = '.', label = 'Out (1,1,1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,1,1])], marker = '.', label = 'Out (-1,1,1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([1,-1,1])], marker = '.', label = 'Out (1,-1,1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,-1,1])], marker = '.', label = 'Out (-1,-1,1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([1,1,-1])], marker = '.', label = 'Out (1,1,-1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,1,-1])], marker = '.', label = 'Out (-1,1,-1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([1,-1,-1])], marker = '.', label = 'Out (1,-1,-1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,-1,-1])], marker = '.', label = 'Out (-1,-1,-1)'),
    Line2D([0], [0], color = 'gray', label = 'Path of output')
]
ax.legend(handles = custom_legend, loc=9, ncol=3)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.view_init(14,-119)

<IPython.core.display.Javascript object>

---
PCA analysis of test for delay of 8 time-steps.

---

# Slow point analysis

0 delay

In [15]:
# Calculating approximate cube
s7d0_center_array = []
for i in range(8):
    indexes = np.where(np.equal(s7d0_out_type,i*np.ones(len(s7d0_out_type))))[0]
    s7d0_center= np.mean(s7d0_states[indexes],axis=0)
    s7d0_center_array.append(s7d0_center)
s7d0_center_array = np.array(s7d0_center_array)

s7d0_cube = s7d0_center_array[switch]

In [16]:
# Finding slow points
s7d0_min_q = []
for i in range(300):
    opt_result = scipy.optimize.minimize(q,(np.random.rand(nb_neurons)-0.5)*10,args=(s7d0_w))
    # if {optimization was a success} and ( {this is the first slow point} or {slow point does not correspond to one already found} )
    if opt_result['success'] and (len(s7d0_min_q)==0 or np.all([np.linalg.norm(np.abs(min_q_i-opt_result['x']))>1 for min_q_i in s7d0_min_q])):
        s7d0_min_q.append(opt_result['x'])
s7d0_min_q = np.array(s7d0_min_q)

# Calculating the number of free dimensions for each slow point
s7d0_q_type = []
for min_q_i in s7d0_min_q:
    J = np.empty((nb_neurons,nb_neurons))
    for i in range(nb_neurons):
        J[i,:] = np.dot(s7d0_w[i,:],(1/np.cosh(min_q_i[i]))**2)-I[i,:]
    s7d0_q_type.append(sum(np.linalg.eig(J)[0]>0))

In [17]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

for edge in s7d0_cube:
    edge_pc1, edge_pc2, edge_pc3 = s7d0_pca.transform(edge).T
    ax.plot(edge_pc1, edge_pc2, edge_pc3, '--', c='darkgray', lw=1)

min_q_pc1, min_q_pc2, min_q_pc3 = s7d0_pca.transform(s7d0_min_q).T
for i in range(s7d0_min_q.shape[0]):
    if(s7d0_q_type[i]<=3):
        ax.scatter(min_q_pc1[i], min_q_pc2[i], min_q_pc3[i], c=['k','b','g','y'][s7d0_q_type[i]], marker='x', alpha=0.7)
    else:
        ax.scatter(min_q_pc1[0,i], min_q_pc2[1,i], min_q_pc3[2,i], c='r', marker='^', alpha=0.7)
        ax.text(min_q_pc1[i], min_q_pc2[i], min_q_pc3[i], str(s7d0_q_type[i]))

custom_legend = [
    Line2D([0], [0], color = 'w', markersize = 10, markerfacecolor = 'k', marker = 'X', label = 'Fixed point'),
    Line2D([0], [0], color = 'w', markersize = 10, markerfacecolor = 'b', marker = 'X', label = 'Saddle point (1 conv dim)'),
    Line2D([0], [0], color = 'w', markersize = 10, markerfacecolor = 'g', marker = 'X', label = 'Saddle point (2 conv dims)'),
    Line2D([0], [0], color = 'w', markersize = 10, markerfacecolor = 'y', marker = 'X', label = 'Saddle point (3 conv dims)'),
    Line2D([0], [0], color = 'darkgray', linestyle='--', label = 'Approximate path')
]
ax.legend(handles = custom_legend, loc=9, ncol=2)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.view_init(4,-8)

<IPython.core.display.Javascript object>

---
Slow point analysis of RNN without delay. For clarity, a cube is used to mark the approximate path of neurons states during the test. The slow points are color coded with respect to the number of convex dimensions.

---

4 delay

In [18]:
# Calculating approximate cube
s7d4_center_array = []
for i in range(8):
    indexes = np.where(np.equal(s7d4_out_type,i*np.ones(len(s7d4_out_type))))[0]
    s7d4_center= np.mean(s7d4_states[indexes],axis=0)
    s7d4_center_array.append(s7d4_center)
s7d4_center_array = np.array(s7d4_center_array)

s7d4_cube = s7d4_center_array[switch]

In [19]:
# Finding slow points
s7d4_min_q = []
for i in range(300):
    opt_result = scipy.optimize.minimize(q,(np.random.rand(nb_neurons)-0.5)*10,args=(s7d4_w))
    # if {optimization was a success} and ( {this is the first slow point} or {slow point does not correspond to one already found} )
    if opt_result['success'] and (len(s7d4_min_q)==0 or np.all([np.linalg.norm(np.abs(min_q_i-opt_result['x']))>1 for min_q_i in s7d4_min_q])):
        s7d4_min_q.append(opt_result['x'])
s7d4_min_q = np.array(s7d4_min_q)

# Calculating the number of free dimensions for each slow point
s7d4_q_type = []
for min_q_i in s7d4_min_q:
    J = np.empty((nb_neurons,nb_neurons))
    for i in range(nb_neurons):
        J[i,:] = np.dot(s7d4_w[i,:],(1/np.cosh(min_q_i[i]))**2)-I[i,:]
    s7d4_q_type.append(sum(np.linalg.eig(J)[0]>0))

In [20]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

for edge in s7d4_cube:
    edge_pc1, edge_pc2, edge_pc3 = s7d4_pca.transform(edge).T
    ax.plot(edge_pc1, edge_pc2, edge_pc3, '--', c='darkgray', lw=1)

min_q_pc1, min_q_pc2, min_q_pc3 = s7d4_pca.transform(s7d4_min_q).T
for i in range(s7d4_min_q.shape[0]):
    if(s7d4_q_type[i]<=3):
        ax.scatter(min_q_pc1[i], min_q_pc2[i], min_q_pc3[i], c=['k','b','g','y'][s7d4_q_type[i]], marker='x', alpha=0.7)
    else:
        ax.scatter(min_q_pc1[i], min_q_pc2[i], min_q_pc3[i], c='r', marker='^', alpha=0.7)
        ax.text(min_q_pc1[i], min_q_pc2[i], min_q_pc3[i], str(s7d4_q_type[i]))

custom_legend = [
    Line2D([0], [0], color = 'w', markersize = 10, markerfacecolor = 'k', marker = 'X', label = 'Fixed point'),
    Line2D([0], [0], color = 'w', markersize = 10, markerfacecolor = 'b', marker = 'X', label = 'Saddle point (1 conv dim)'),
    Line2D([0], [0], color = 'w', markersize = 10, markerfacecolor = 'g', marker = 'X', label = 'Saddle point (2 conv dims)'),
    Line2D([0], [0], color = 'w', markersize = 10, markerfacecolor = 'y', marker = 'X', label = 'Saddle point (3 conv dims)'),
    Line2D([0], [0], color = 'darkgray', linestyle='--', label = 'Approximate path')
]
ax.legend(handles = custom_legend, loc=9, ncol=2)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.view_init(3,83)

<IPython.core.display.Javascript object>

---
Slow point analysis for delay of 4 time-steps.

---

8 delay

In [21]:
# Calculating approximate cube
s7d8_center_array = []
for i in range(8):
    indexes = np.where(np.equal(s7d8_out_type,i*np.ones(len(s7d8_out_type))))[0]
    s7d8_center= np.mean(s7d8_states[indexes],axis=0)
    s7d8_center_array.append(s7d8_center)
s7d8_center_array = np.array(s7d8_center_array)

s7d8_cube = s7d8_center_array[switch]

In [ ]:
# Finding slow points
s7d8_min_q = []
for i in range(500):
    opt_result = scipy.optimize.minimize(q,(np.random.rand(nb_neurons)-0.5)*10,args=(s7d8_w))
    # if {optimization was a success} and ( {this is the first slow point} or {slow point does not correspond to one already found} )
    if opt_result['success'] and (len(s7d8_min_q)==0 or np.all([np.linalg.norm(np.abs(min_q_i-opt_result['x']))>0.2 for min_q_i in s7d8_min_q])):
        s7d8_min_q.append(opt_result['x'])
s7d8_min_q = np.array(s7d8_min_q)

# Calculating the number of free dimensions for each slow point
s7d8_q_type = []
for min_q_i in s7d8_min_q:
    J = np.empty((nb_neurons,nb_neurons))
    for i in range(nb_neurons):
        J[i,:] = np.dot(s7d8_w[i,:],(1/np.cosh(min_q_i[i]))**2)-I[i,:]
    s7d8_q_type.append(sum(np.linalg.eig(J)[0]>0))

# This might be really long.................................................................................................................................................................................................................................. So... how was your day?........

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

for edge in s7d8_cube:
    edge_pc1, edge_pc2, edge_pc3 = s7d8_pca.transform(edge).T
    ax.plot(edge_pc1, edge_pc2, edge_pc3, '--', c='darkgray', lw=1)

min_q_pc1, min_q_pc2, min_q_pc3 = s7d8_pca.transform(s7d8_min_q).T
for i in range(s7d8_min_q.shape[0]):
    if(s7d8_q_type[i]<=3):
        ax.scatter(min_q_pc1[i], min_q_pc2[i], min_q_pc3[i], c=['k','b','g','y'][s7d8_q_type[i]], marker='x', alpha=0.7)
    else:
        ax.scatter(min_q_pc1[i], min_q_pc2[i], min_q_pc3[i], c='r', marker='^', alpha=0.7)
        ax.text(min_q_pc1[i], min_q_pc2[i], min_q_pc3[i], str(s7d8_q_type[i]))

custom_legend = [
    Line2D([0], [0], color = 'w', markersize = 10, markerfacecolor = 'k', marker = 'X', label = 'Fixed point'),
    Line2D([0], [0], color = 'w', markersize = 10, markerfacecolor = 'b', marker = 'X', label = 'Saddle point (1 conv dim)'),
    Line2D([0], [0], color = 'w', markersize = 10, markerfacecolor = 'g', marker = 'X', label = 'Saddle point (2 conv dims)'),
    Line2D([0], [0], color = 'w', markersize = 10, markerfacecolor = 'y', marker = 'X', label = 'Saddle point (3 conv dims)'),
    Line2D([0], [0], color = 'w', markersize = 10, markerfacecolor = 'r', marker = '^', label = 'Saddle point (>3 conv dims)'),
    Line2D([0], [0], color = 'darkgray', linestyle='--', label = 'Approximate path')
]
ax.legend(handles = custom_legend, loc=9, ncol=2)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.view_init(3,-81)

---
Slow point analysis for delay of 8 time-steps. There are now slow points with more than three convex dimensions. Those are marked by red triangles and the number of convex dimensions is written above.

---

# Triggered average analysis

0 delay

In [ ]:
time_delay = 0
nb_before = time_delay+10
nb_after = 11
pattern = np.concatenate((np.ones(nb_before)*tension_to_index([-1,-1,-1]),np.ones(nb_after+time_delay)*tension_to_index([-1,-1,1]))).astype('int')
triggered_indexes = matchPattern(s7d0_out_type,pattern)
triggered_indexes = np.array([np.arange(i,i+nb_before+nb_after) for i in triggered_indexes])
s7d0_triggered_states = np.mean(s7d0_states[triggered_indexes],axis=0)

In [ ]:
s7d0_triggered_pca = PCA(3)
s7d0_triggered_pca.fit(s7d0_triggered_states)
s7d0_trig_pc1, s7d0_trig_pc2, s7d0_trig_pc3 = s7d0_triggered_pca.transform(s7d0_triggered_states).T

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

for edge in s7d0_cube:
    edge_pc1, edge_pc2, edge_pc3 = s7d0_triggered_pca.transform(edge).T
    ax.plot(edge_pc1, edge_pc2, edge_pc3, '--', c='darkgray', lw=1)

for i in range(nb_before+nb_after):
    ax.scatter(s7d0_trig_pc1[i], s7d0_trig_pc2[i], s7d0_trig_pc3[i], c=colors[pattern[i]], alpha=0.3)
ax.plot(s7d0_trig_pc1, s7d0_trig_pc2, s7d0_trig_pc3, '-k', linewidth=0.5, alpha=0.3)

custom_legend = [
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,-1,-1])], marker = '.', label = 'Out (-1,-1,-1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,-1,1])], marker = '.', label = 'Out (-1,-1,1)'),
    Line2D([0], [0], color = 'gray', label = 'Path of output'),
    Line2D([0], [0], color = 'darkgray', linestyle='--', label = 'Approximate path')
]
ax.legend(handles = custom_legend, loc=9, ncol=2)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.view_init(11,146)

---
The switch from output (-1,-1,-1) to output (-1,-1,1) for the RNN without delay was studied using a triggered average. Here is the PCA analysis of the transition state. A few points before and after the switch are also plotted to show the relative movement of the transition to that of the two quasi-stable states.

---

4 delay

In [ ]:
time_delay = 4
nb_before = time_delay+10
nb_after = 11
pattern = np.concatenate((np.ones(nb_before)*tension_to_index([-1,-1,-1]),np.ones(nb_after+time_delay)*tension_to_index([-1,-1,1]))).astype('int')
triggered_indexes = matchPattern(s7d4_out_type,pattern)
triggered_indexes = np.array([np.arange(i,i+nb_before+nb_after) for i in triggered_indexes])
s7d4_triggered_states = np.mean(s7d4_states[triggered_indexes],axis=0)

In [ ]:
s7d4_triggered_pca = PCA(3)
s7d4_triggered_pca.fit(s7d4_triggered_states)
s7d4_trig_pc1, s7d4_trig_pc2, s7d4_trig_pc3 = s7d4_triggered_pca.transform(s7d4_triggered_states).T

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

for edge in s7d4_cube:
    edge_pc1, edge_pc2, edge_pc3 = s7d4_triggered_pca.transform(edge).T
    ax.plot(edge_pc1, edge_pc2, edge_pc3, '--', c='darkgray', lw=1)

for i in range(nb_before+nb_after):
    ax.scatter(s7d4_trig_pc1[i], s7d4_trig_pc2[i], s7d4_trig_pc3[i], c=colors[pattern[i]], alpha=0.3)
ax.plot(s7d4_trig_pc1, s7d4_trig_pc2, s7d4_trig_pc3, '-k', linewidth=0.5, alpha=0.3)

custom_legend = [
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,-1,-1])], marker = '.', label = 'Out (-1,-1,-1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,-1,1])], marker = '.', label = 'Out (-1,-1,1)'),
    Line2D([0], [0], color = 'gray', label = 'Path of output'),
    Line2D([0], [0], color = 'darkgray', linestyle='--', label = 'Approximate path')
]
ax.legend(handles = custom_legend, loc=9, ncol=2)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.view_init(46,-150)

---
The switch from output (-1,-1,-1) to output (-1,-1,1) for delay of 4 time-steps.

---

8 delay

In [ ]:
time_delay = 8
nb_before = time_delay+10
nb_after = 11
pattern = np.concatenate((np.ones(nb_before)*tension_to_index([-1,-1,-1]),np.ones(nb_after+time_delay)*tension_to_index([-1,-1,1]))).astype('int')
triggered_indexes = matchPattern(s7d8_out_type,pattern)
triggered_indexes = np.array([np.arange(i,i+nb_before+nb_after) for i in triggered_indexes])
s7d8_triggered_states = np.mean(s7d8_states[triggered_indexes],axis=0)

In [ ]:
s7d8_triggered_pca = PCA(3)
s7d8_triggered_pca.fit(s7d8_triggered_states)
s7d8_trig_pc1, s7d8_trig_pc2, s7d8_trig_pc3 = s7d8_triggered_pca.transform(s7d8_triggered_states).T

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

for edge in s7d8_cube:
    edge_pc1, edge_pc2, edge_pc3 = s7d8_triggered_pca.transform(edge).T
    ax.plot(edge_pc1, edge_pc2, edge_pc3, '--', c='darkgray', lw=1)

for i in range(nb_before+nb_after):
    ax.scatter(s7d8_trig_pc1[i], s7d8_trig_pc2[i], s7d8_trig_pc3[i], c=colors[pattern[i]], alpha=0.3)
ax.plot(s7d8_trig_pc1, s7d8_trig_pc2, s7d8_trig_pc3, '-k', linewidth=0.5, alpha=0.3)

custom_legend = [
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,-1,-1])], marker = '.', label = 'Out (-1,-1,-1)'),
    Line2D([0], [0], color = 'w', markersize = 15, markerfacecolor = colors[tension_to_index([-1,-1,1])], marker = '.', label = 'Out (-1,-1,1)'),
    Line2D([0], [0], color = 'gray', label = 'Path of output'),
    Line2D([0], [0], color = 'darkgray', linestyle='--', label = 'Approximate path')
]
ax.legend(handles = custom_legend, loc=9, ncol=2)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.view_init(6,152)

---
The switch from output (-1,-1,-1) to output (-1,-1,1) for delay of 8 time-steps.

---

In [ ]:
s7d0_triggered_pca_long = PCA(4)
s7d0_triggered_pca_long.fit(s7d0_triggered_states)
s7d0_output_array = np.tanh(np.matmul(s7d0_triggered_states,s7d0_wout))

fig, ax_array = plt.subplots(6,1,sharex=True,figsize=(8,8))
fig.subplots_adjust(hspace=0)

trig_states = s7d0_triggered_pca_long.transform(s7d0_triggered_states).T
input_array = np.zeros(21)
input_array[10] = -1
output_array = s7d0_output_array
lim = np.max(np.abs(trig_states))

for ax in ax_array:
    ax.grid(True)
    ax.get_xaxis().set_ticks([])
    ax.axvline(0,linestyle='--',color='gray',linewidth=1)

for j in range(4):
    ax_array[j].plot(np.arange(-10,11),trig_states[j],'k-',linewidth=1)
    ax_array[j].plot(np.arange(-10,11),trig_states[j],'b.',markersize=5)
    ax_array[j].get_yaxis().set_ticks([-2,0,2])
    ax_array[j].set_ylim(-1.2*lim,1.2*lim)
    ax_array[j].set_ylabel('PC '+str(j),fontsize=10)

ax_array[4].plot(np.arange(-10,11),output_array[:,2],'-k')
ax_array[4].get_yaxis().set_ticks([])
ax_array[4].set_ylabel('Output ch 3',fontsize=10)

ax_array[5].plot(np.arange(-10,11),input_array,'-k')
ax_array[5].get_yaxis().set_ticks([])
ax_array[5].set_ylabel('Input ch 3',fontsize=10)

ax_array[5].set_xlabel('Elapsed time-steps')

In [ ]:
s7d4_triggered_pca_long = PCA(7)
s7d4_triggered_pca_long.fit(s7d4_triggered_states)
s7d4_output_array = np.tanh(np.matmul(s7d4_triggered_states,s7d4_wout))

fig, ax_array = plt.subplots(9,1,sharex=True,figsize=(8,12))
fig.subplots_adjust(hspace=0)

trig_states = s7d4_triggered_pca_long.transform(s7d4_triggered_states).T
input_array = np.zeros(21+4)
input_array[10] = -1
output_array = s7d4_output_array
lim = np.max(np.abs(trig_states))

for ax in ax_array:
    ax.grid(True)
    ax.get_xaxis().set_ticks([])
    ax.axvline(0,linestyle='--',color='gray',linewidth=1)
    ax.axvline(4,linestyle='--',color='gray',linewidth=1)

for j in range(7):
    ax_array[j].plot(np.arange(-10,11+4),trig_states[j],'k-',linewidth=1)
    ax_array[j].plot(np.arange(-10,11+4),trig_states[j],'b.',markersize=5)
    ax_array[j].get_yaxis().set_ticks([-2,0,2])
    ax_array[j].set_ylim(-1.2*lim,1.2*lim)
    ax_array[j].set_ylabel('PC '+str(j),fontsize=10)

ax_array[7].plot(np.arange(-10,11+4),output_array[:,2],'-k')
ax_array[7].get_yaxis().set_ticks([])
ax_array[7].set_ylabel('Output channel 3',fontsize=5)

ax_array[8].plot(np.arange(-10,11+4),input_array,'-k')
ax_array[8].get_yaxis().set_ticks([])
ax_array[8].set_ylabel('Input channel 3',fontsize=5)

ax_array[8].set_xlabel('Elapsed time-steps')

In [ ]:
s7d8_triggered_pca_long = PCA(9)
s7d8_triggered_pca_long.fit(s7d8_triggered_states)
s7d8_output_array = np.tanh(np.matmul(s7d8_triggered_states,s7d8_wout))

fig, ax_array = plt.subplots(11,1,sharex=True,figsize=(8,15))
fig.subplots_adjust(hspace=0)

trig_states = s7d8_triggered_pca_long.transform(s7d8_triggered_states).T
input_array = np.zeros(21+8)
input_array[10] = -1
output_array = s7d8_output_array
lim = np.max(np.abs(trig_states))

for ax in ax_array:
    ax.grid(True)
    ax.get_xaxis().set_ticks([])
    ax.axvline(0,linestyle='--',color='gray',linewidth=1)
    ax.axvline(8,linestyle='--',color='gray',linewidth=1)

for j in range(9):
    ax_array[j].plot(np.arange(-10,11+8),trig_states[j],'k-',linewidth=1)
    ax_array[j].plot(np.arange(-10,11+8),trig_states[j],'b.',markersize=5)
    ax_array[j].get_yaxis().set_ticks([-2,0,2])
    ax_array[j].set_ylim(-1.2*lim,1.2*lim)
    ax_array[j].set_ylabel('PC '+str(j),fontsize=10)

ax_array[9].plot(np.arange(-10,11+8),output_array[:,0],'-k')
ax_array[9].get_yaxis().set_ticks([])
ax_array[9].set_ylabel('Output channel 3',fontsize=5)

ax_array[10].plot(np.arange(-10,11+8),input_array,'-k')
ax_array[10].get_yaxis().set_ticks([])
ax_array[10].set_ylabel('Input channel 3',fontsize=5)

ax_array[10].set_xlabel('Elapsed time-steps')

# Eigenvalues analysis

In [ ]:
s7d0_eigvals = np.linalg.eig(s7d0_w)[0]
s7d4_eigvals = np.linalg.eig(s7d4_w)[0]
s7d8_eigvals = np.linalg.eig(s7d8_w)[0]

fig, (ax0,ax1,ax2) = plt.subplots(1, 3, sharey=True, figsize=(8,3))

ax0.axvline(0,linestyle='--',color='gray',linewidth=1)
ax0.axhline(0,linestyle='--',color='gray',linewidth=1)
ax0.plot(s7d0_eigvals.real,s7d0_eigvals.imag,'b+',ms=5)
ax0.set_xticks([-0.5,0,0.5,1])

ax1.axvline(0,linestyle='--',color='gray',linewidth=1)
ax1.axhline(0,linestyle='--',color='gray',linewidth=1)
ax1.plot(s7d4_eigvals.real,s7d4_eigvals.imag,'b+',ms=5)
ax1.set_xticks([-1,-0.5,0,0.5,1,1.5])

ax2.axvline(0,linestyle='--',color='gray',linewidth=1)
ax2.axhline(0,linestyle='--',color='gray',linewidth=1)
ax2.plot(s7d8_eigvals.real,s7d8_eigvals.imag,'b+',ms=5)
ax2.set_xticks([-1,0,1,2])

ax0.set_ylabel('Complex value')
ax1.set_xlabel('Real value')

fig.tight_layout()

---
Analysis of the eigenvalues for the three studied RNNs. The eigenvalues of the network without delay is diplayed to the left, the ones for 4 time-steps at the center and the ones for 8 time-steps to the right.

---